In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Veo 2 Editing

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb">
      <img src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fvision%2Fgetting-started%2Fveo2_editing.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/vision/getting-started/veo2_editing.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
|Author(s) | [Katie Nguyen](https://github.com/katiemn) |

**Launch Stage: Experimental**

## Overview

### Veo 2

Veo 2 on Vertex AI brings Google's video generation capabilities to application developers. It's capable of creating videos with astonishing detail that simulate real-world physics across a wide range of visual styles.

In this tutorial, you will learn how to use the Vertex AI API to interact with Veo 2 Editing to:
- Inpaint
- Outpaint

## Get started

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# fmt: on

### Import libraries

In [ ]:
import time

import google.auth
import google.auth.transport.requests
import requests
from IPython.display import Image, Video, display

### Define helper functions

In [ ]:
def send_request_to_google_api(api_endpoint, data=None):
    """Sends an HTTP request to a Google API endpoint.

    Args:
        api_endpoint: The URL of the Google API endpoint.
        data: (Optional) Dictionary of data to send in the request body (for POST, PUT, etc.).

    Returns:
        The response from the Google API.
    """
    # Get access token calling API
    creds, project = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)
    access_token = creds.token

    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }

    response = requests.post(api_endpoint, headers=headers, json=data)
    response.raise_for_status()
    return response.json()


def compose_request(
    prompt,
    parameters: dict[str, object],
    mask_gcs: str = "",
    mask_mime_type: str = "",
    mask_mode: str = "",
    video_gcs: str = "",
):
    instance = {"prompt": prompt}
    if mask_gcs:
        instance["mask"] = {
            "gcsUri": mask_gcs,
            "mimeType": mask_mime_type,
            "maskMode": mask_mode,
        }
    if video_gcs:
        instance["video"] = {"gcsUri": video_gcs, "mimeType": "video/mp4"}
    request = {"instances": [instance], "parameters": parameters}
    return request


def fetch_operation(lro_name):
    request = {"operationName": lro_name}
    for i in range(30):
        resp = send_request_to_google_api(fetch_endpoint, request)
        if resp.get("done"):
            return resp
        time.sleep(10)


def generate_video(
    prompt: str,
    parameters: dict[str, object],
    mask_gcs: str = "",
    mask_mime_type: str = "",
    mask_mode: str = "",
    video_gcs: str = "",
):
    req = compose_request(
        prompt=prompt,
        parameters=parameters,
        mask_gcs=mask_gcs,
        mask_mime_type=mask_mime_type,
        mask_mode=mask_mode,
        video_gcs=video_gcs,
    )
    resp = send_request_to_google_api(prediction_endpoint, req)
    print(resp)
    return fetch_operation(resp["name"])


def show_video(op):
    if "error" in op:
        print("\n" + op["error"]["message"])
        return
    if "videos" in op["response"]:
        for video in op["response"]["videos"]:
            gcs_uri = video["gcsUri"]
            show_video_from_gcs(gcs_uri)


def show_video_from_gcs(video_gcs):
    file_name = video_gcs.split("/")[-1]
    !gsutil cp {video_gcs} {file_name}
    display(Video(file_name, embed=True, height=400))


def show_image_from_gcs(image_gcs):
    file_name = image_gcs.split("/")[-1]
    !gsutil cp {image_gcs} {file_name}
    display(Image(file_name, height=400))

### Load the video model

In [ ]:
video_model = f"https://us-central1-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/us-central1/publishers/google/models/veo-2.0-generate-exp"
prediction_endpoint = f"{video_model}:predictLongRunning"
fetch_endpoint = f"{video_model}:fetchPredictOperation"

## Inpainting

With inpainting, you can provide a mask to remove objects from a video. This can be done through static or dynamic inpainting.

### Static inpainting
Static inpainting applies the given mask image to each frame of the video, allowing you to mask out sections of a video and remove content.

This example will walk you through using an original video and mask image that's stored in Cloud Storage. In order to generate an edited video, specify the following:

- **Prompt:** Can be empty since you're removing content from the video.
- **Video GCS:** The Cloud Storage location of your starting video.
- **Mask GCS:** The Cloud Storage location of your image mask.
- **Mask mime type:** Select either `image/png`, `image/jpeg`, `video/mp4`.
- **Output file location:** The generated video will be shown below with support from a previously defined helper function. The video will also be stored in Cloud Storage once video generation is complete. Specify the bucket path where you would like this video to be stored in `output_gcs`.
- **Aspect ratio:** Select either 16:9 or 9:16.
- **Prompt enhancement:** The model offers the option to enhance your provided prompt. To utilize this feature, set `enhance_prompt` to True. A new, detailed prompt will be created from your original one to help generate higher quality videos that better adhere to your prompt's intent.


In [ ]:
prompt = ""  # @param {type: 'string'}
# fmt: off
video_gcs = "gs://cloud-samples-data/generative-ai/video/truck.mp4"  # @param {type: 'string'}
mask_gcs = "gs://cloud-samples-data/generative-ai/image/truck-inpainting-static-mask.png"  # @param {type: 'string'}
# fmt: on
mask_mime_type = "image/png"  # @param ["image/png", 'image/jpeg', 'video/mp4']

output_gcs = "gs://[your-bucket-path]"  # @param {type: 'string'}
aspect_ratio = "16:9"  # @param ["16:9", "9:16"]
enhance_prompt = False  # @param {type: 'boolean'}
sample_count = 1  # @param {type: 'number'}
duration = 8  # @param {type: 'number'}

parameters = {
    "storageUri": output_gcs,
    "aspectRatio": aspect_ratio,
    "enhancePrompt": enhance_prompt,
    "sampleCount": sample_count,
    "durationSeconds": duration,
}

print("----Original Video----")
show_video_from_gcs(video_gcs)
print("----Image Mask----")
show_image_from_gcs(mask_gcs)

op = generate_video(
    prompt,
    video_gcs=video_gcs,
    mask_gcs=mask_gcs,
    mask_mime_type=mask_mime_type,
    mask_mode="INPAINT",
    parameters=parameters,
)

print("----Edited Video----")
show_video(op)

### Dynamic inpainting
Dynamic inpainting uses the given mask image to select an object from the first frame to remove. Only one object is supported at a time.

This example will walk you through using an original video and mask image that's stored in Cloud Storage. Since you're removing content from a video you don't need to specify a prompt.

In [ ]:
# fmt: off
video_gcs = "gs://cloud-samples-data/generative-ai/video/truck.mp4"  # @param {type: 'string'}
mask_gcs = "gs://cloud-samples-data/generative-ai/image/truck-inpainting-dynamic-mask.png"  # @param {type: 'string'}
# fmt: on
mask_mime_type = "image/png"  # @param ["image/png", 'image/jpeg', 'video/mp4']

output_gcs = "gs://[your-bucket-path]"  # @param {type: 'string'}
aspect_ratio = "16:9"  # @param ["16:9", "9:16"]
sample_count = 1  # @param {type: 'number'}
duration = 8  # @param {type: 'number'}

parameters = {
    "storageUri": output_gcs,
    "aspectRatio": aspect_ratio,
    "sampleCount": sample_count,
    "durationSeconds": duration,
}

print("----Original Video----")
show_video_from_gcs(video_gcs)
print("----Image Mask----")
show_image_from_gcs(mask_gcs)

op = generate_video(
    prompt="",
    video_gcs=video_gcs,
    mask_gcs=mask_gcs,
    mask_mime_type=mask_mime_type,
    mask_mode="INPAINT_DYNAMIC",
    parameters=parameters,
)

print("----Edited Video----")
show_video(op)

## Outpainting
In this next example, you'll edit a video with Veo 2 by outpainting. This feature entails extending a video. To outpaint, provide an image mask with a single, unmasked rectangle representing where the original video should be placed.

This example will walk you through using an original video and mask image that's stored in Cloud Storage. You can add a prompt if you would like certain content in the outpainted area, or you can leave the prompt field empty.

**Safety:** All Veo videos include [SynthID](https://deepmind.google/technologies/synthid/).

In [ ]:
prompt = ""  # @param {type: 'string'}
# fmt: off
video_gcs = "gs://cloud-samples-data/generative-ai/video/truck.mp4"  # @param {type: 'string'}
mask_gcs = "gs://cloud-samples-data/generative-ai/image/truck-outpainting-mask.png"  # @param {type: 'string'}
# fmt: on
mask_mime_type = "image/png"  # @param ["image/png", 'image/jpeg', 'video/mp4']

output_gcs = "gs://[your-bucket-path]"  # @param {type: 'string'}
aspect_ratio = "16:9"  # @param ["16:9", "9:16"]
enhance_prompt = False  # @param {type: 'boolean'}
sample_count = 1  # @param {type: 'number'}
duration = 8  # @param {type: 'number'}

parameters = {
    "storageUri": output_gcs,
    "aspectRatio": aspect_ratio,
    "enhancePrompt": enhance_prompt,
    "sampleCount": sample_count,
    "durationSeconds": duration,
}

print("----Original Video----")
show_video_from_gcs(video_gcs)
print("----Image Mask----")
show_image_from_gcs(mask_gcs)

op = generate_video(
    prompt,
    video_gcs=video_gcs,
    mask_gcs=mask_gcs,
    mask_mime_type=mask_mime_type,
    mask_mode="OUTPAINT",
    parameters=parameters,
)

print("----Edited Video----")
show_video(op)